# **Step1_AI면접관 Agent v1.0**

### (5) 인터뷰 피드백 보고서
* 목적 : 인터뷰를 종료하며, 인터뷰 내용과 평가내용 화면 출력
* 입출력
    * 입력 : State
    * 출력 : State
* 처리 : 질문 / 답변 / 평가에 대한 순차적 출력


In [ ]:
# 기존 코드

# def summarize_interview(state: InterviewState) -> InterviewState:
#     # 여기에 코드를 완성합니다.

#     conversation_history = state["conversation"]
#     evaluate_history = state["evaluation"]

#     print("인터뷰 종료 리포트")
#     print("="*150)
#     for i in range(len(conversation_history)):
#       print(f"[질문 {i+1}] {conversation_history[i]['질문']}")
#       print(f"[답변 {i+1}] {conversation_history[i]['답변']}")
#       print(f"[평가] 질문과의 관련성: {evaluate_history[i]['질문과의 관련성']}, 답변의 구체성: {evaluate_history[i]['답변의 구체성']}")
#       print("="*150)

#     # return 코드는 제공합니다.
#     return state

In [ ]:
def summarize_interview(state: InterviewState) -> InterviewState:
    # 전략 분류 목록
    strategy_categories = [
        "경력 및 경험",
        "동기",
        "커뮤니케이션",
        "논리적 사고",
        "기술 스택",
        "장단점"
    ]

    categorized_data = {category: [] for category in strategy_categories}

    conversation = state["conversation"]
    evaluation = state["evaluation"]

    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.7)

    # (1) 질문 전략을 그대로 활용하여 분류
    for i, qa in enumerate(conversation):
        question = qa["질문"]
        answer = qa["답변"]
        strategy = qa.get("질문 전략")

        categorized_data[strategy].append((question, answer, evaluation[i]))

    # (2) 전략별 피드백 생성 (요약 / 강점 / 약점)
    strategy_feedback = {}
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", "아래 인터뷰 질문과 답변을 바탕으로 답변 요약, 강점, 약점을 작성해줘. 포맷:\n- 답변 요약:\n- 강점:\n- 약점:\n"),
        ("human", "{qna_text}")
    ])

    # strategy_feedback[category]에 결과 저장
    for category, items in categorized_data.items():
        if not items:
            continue
        qna_text = "\n".join(
            f"Q: {q}\nA: {a}" for q, a, _ in items
        )
        feedback_msg = feedback_prompt.format_messages(qna_text=qna_text)
        feedback_result = llm(feedback_msg).content.strip()
        strategy_feedback[category] = feedback_result

    # (3) 종합 피드백 생성
    overall_prompt = ChatPromptTemplate.from_messages([
        ("system", "아래 면접 기록을 바탕으로 종합 피드백을 작성해줘. 강점과 개선점을 중심으로 작성하고, 격려하는 말투로 마무리해. 모든 출력은 **와 같은 마크다운 문법 없이 일반 텍스트로 작성해줘."),
        ("human", "{full_text}")
    ])
    merged_text = "\n".join([
        f"[{category}]\n{feedback}" for category, feedback in strategy_feedback.items()
    ])
    overall_msg = overall_prompt.format_messages(full_text=merged_text)
    overall_feedback = llm(overall_msg).content.strip()

    # (4) 출력
    print("\n[인터뷰 피드백 보고서]")
    print("-" * 80)
    for category in strategy_categories:
        if category in strategy_feedback:
            print(f"\n[{category}]")
            print(strategy_feedback[category])
            print("-" * 80)

    print("\n[종합 피드백]")
    print(overall_feedback)
    print("-" * 80)


    return state


In [ ]:
test_state["conversation"] = [
        {
            "질문": "전기정보공학 배경을 바탕으로 금융 서비스 개발에 참여한 경험이 있다면, 그 과정에서 어떤 문제를 해결했는지 구체적으로 설명해줄 수 있나요?",
            "답변": "저는 해당 프로젝트에서 PM 역할을 맡아 팀을 조율하며 마감기한 내에 서비스를 성공적으로 출시했습니다.",
            "질문 전략": "경력 및 경험"
        },
        {
            "질문": "AI 면접관 시스템을 졸업 과제로 개발한 경험이 있다고 하였다고 했는데, 해당 프로젝트에 대한 설명과 그 과정에서의 주요 어려움은 무엇이었는지 말씀해 주세요.",
            "답변": "기억이 잘 나지 않습니다..",
            "질문 전략": "경력 및 경험"
        },
        {
            "질문": "기획자와의 의견 차이가 있었을 때 어떻게 대응하셨나요?",
            "답변": "일정을 조율하며 서로의 입장을 고려해 우선순위를 재정의하는 방식으로 해결했습니다.",
            "질문 전략": "커뮤니케이션"
        },
        {
            "질문": "프로젝트에서 문제가 발생했을 때, 어떤 방식으로 원인을 파악하고 해결하셨나요?",
            "답변": "로그 분석과 데이터 시각화를 통해 문제를 진단하고 팀원들과 함께 해결 방안을 도출했습니다.",
            "질문 전략": "논리적 사고"
        },
        {
            "질문": "어떤 기술 스택을 가장 자신 있게 다룰 수 있나요?",
            "답변": "저는 Python과 pandas, SQL에 익숙하고 Flask로 간단한 API 서버를 구축한 경험이 있습니다.",
            "질문 전략": "기술 스택"
        },
        {
            "질문": "본인의 장점과 단점을 말씀해 주세요.",
            "답변": "저는 끈기와 책임감이 강한 편이며, 단점은 가끔 완벽주의 성향으로 일정이 늦어질 수 있다는 점입니다.",
            "질문 전략": "장단점"
        }
    ]


In [ ]:
test_state["evaluation"] = [
    {"질문과의 관련성": "상", "답변의 구체성": "상"},
    {"질문과의 관련성": "하", "답변의 구체성": "하"},
    {"질문과의 관련성": "상", "답변의 구체성": "상"},
    {"질문과의 관련성": "상", "답변의 구체성": "상"},
    {"질문과의 관련성": "상", "답변의 구체성": "상"},
    {"질문과의 관련성": "하", "답변의 구체성": "중"}
]


In [ ]:
result = summarize_interview(test_state)
print(result)


[인터뷰 피드백 보고서]
--------------------------------------------------------------------------------

[경력 및 경험]
- 답변 요약: 지원자는 전기정보공학 배경을 활용해 금융 서비스 개발 프로젝트에서 PM 역할을 수행하며 팀 조율 및 마감기한 내 서비스 출시를 성공적으로 이끌었다고 설명하였으나, 졸업 과제로 진행한 AI 면접관 시스템에 대한 질문에는 기억이 잘 나지 않는다고 답변했다. 
- 강점: 프로젝트 관리 경험과 팀 조율 능력, 마감 기한 내 서비스 출시 성공.
- 약점: 졸업 과제에 대한 기억 부족으로 인한 구체적인 경험 설명의 어려움.
--------------------------------------------------------------------------------

[커뮤니케이션]
- 답변 요약: 기획자와의 의견 차이를 조율하기 위해 일정을 조정하고 서로의 입장을 고려하여 우선순위를 재정의하는 방법으로 문제를 해결했다.
- 강점: 문제 해결 능력, 협업 및 소통 능력, 유연한 일정 조정 능력.
- 약점: 특정한 상황에서 의견 차이를 해결하기 위한 추가적인 전략이나 구체적인 예시 부족.
--------------------------------------------------------------------------------

[논리적 사고]
- 답변 요약: 프로젝트에서 문제가 발생했을 때, 로그 분석과 데이터 시각화를 활용하여 문제를 진단하고, 팀원들과 협력하여 해결 방안을 찾았다.
- 강점: 문제 해결을 위한 체계적인 접근 방식(로그 분석 및 데이터 시각화)과 협업 능력.
- 약점: 특정한 문제 해결 사례나 구체적인 결과에 대한 언급이 부족하여 경험의 깊이를 확인하기 어려움.
--------------------------------------------------------------------------------

[기술 스택]
- 답변 요약: 지원자는 

In [ ]:
test_state = preProcessing_Interview(file_path)
test_state = update_current_answer(test_state, '저는 해당 프로젝트에서 PM을 맡아서, 잘 진행해봤습니다.')
test_state = evaluate_answer(test_state)
#test_state = generate_question(test_state)
t=summarize_interview(test_state)

<ipython-input-79-020d3a148f4a>:1: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  test_state = preProcessing_Interview(path)
<ipython-input-79-020d3a148f4a>:1: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  test_state = preProcessing_Interview(path)
/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


인터뷰 종료 리포트
[질문 1] 전기정보공학 전공자로서 알고리즘 문제 해결 경험이 금융 서비스 개발에 어떻게 기여할 수 있을지 구체적인 사례를 들어 설명해 주실 수 있나요?
[답변 1] 저는 해당 프로젝트에서 PM을 맡아서, 잘 진행해봤습니다.
[평가] 질문과의 관련성: '하', 답변의 구체성: '하'
